In [16]:
import copy 
import os 
import shutil
import itertools
from collections import Counter
import json
import pickle
import pprint
import pandas as pd
import numpy as np

In [17]:
import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = [6, 8]

In [18]:
print(os.listdir(os.getcwd()))

['.DS_Store', '.ipynb_checkpoints', '0226_pre-standard', '0409_adult-alldata', '0409_german-alldata', '0410_adult-nodata', '0410_german-nodata', '0413_reddata1000', '0501_reddata10000', '0501_reddata2000', '0501_reddata20000', '0501_reddata5000', '0505_reddata1-20t', 'analysis.ipynb', 'backwards_compatibility', 'plotting.ipynb', 'processed_results']


In [19]:
res_dir = '0505_reddata1-20t'
extra = ''
expdir = os.path.join(os.path.join(os.getcwd(), res_dir), 'causal_discovery')
savedir = os.path.join(os.path.join(os.getcwd(), res_dir), '{}_{}'.format(res_dir, extra))

if os.path.exists(savedir):
    shutil.rmtree(savedir)
os.mkdir(savedir)



# Utility Functions 

In [20]:
def dataset_name_from_unid(uid):
    if 'adult' in uid:
        return 'adult'
    if 'german' in uid:
        return 'germanCredit'
    
    assert True == False 

In [21]:
def get_hps_from_rawres(fname):
    '''rawres fname -> features'''
    unique_id = (fname.split('rawres_')[1]).split('.json')[0]
    alpha = unique_id.split('_')[0]
    feateng = unique_id.split('_')[1]
    dataset = unique_id.split('_')[2]
    redsize = unique_id.split('_')[3]
    seed = unique_id.split('_')[4]
    environment = unique_id.split('_')[5]
    
    return feateng, dataset, seed, environment, redsize

In [22]:
def str_2_pcp(pcpstr):
    pcpstr = (pcpstr.split('(')[1]).split(')')[0]
    pcpstr = pcpstr.replace(' ', '')
    ret = set(pcpstr.split(','))
    ret.discard('')
    return ret

In [53]:
import enum 
#Part 1
START_ALPHA = 1.0
FACTOR = 2
EPS = 1e-20
#Part 2
STEP = 1e-2
FACTOR2 = 2
EPS2 = 1e-10

class POS(enum.Enum):
   big = 1
   small = 2
   perf = 3

def alpha_tune(pVals, amin, flag=0):
    #First find a CP returning alpha 
    a0 = START_ALPHA
    bounds0 = [0, 100.0]
    cp_ret = False 
    while not cp_ret:
        pos = 0
        accepted = pVals[pVals['Final_tstat'] > a0]
        
        #Determine position of alpha 
        if len(accepted.index) == 0:
            pos = POS.big
        else: 
            accepted_sets = [str_2_pcp(a) for a in list(accepted.index)]
            causal_preds = set.intersection(*accepted_sets)
            if len(causal_preds) == 0:
                pos = POS.small 
            else:
                pos = POS.perf
                cp_ret = True
                
                if flag:
                    print(causal_preds)
                    print(a0)
                
                continue
                
        #Determine what alpha to check next 
        if pos == POS.big:
            bounds0[1] = a0
            if a0/FACTOR <= bounds0[0]:
                a0 = a0 - abs((a0 - bounds0[0])/2)
            else:
                a0 = a0/FACTOR
        elif pos == POS.small:
            bounds0[0] = a0
            if a0 * FACTOR >= bounds0[1]:
                a0 = a0 + abs((a0 - bounds0[1])/2)
            else:
                a0 = a0 * FACTOR
        
        #Stability check in case no CPs 
        if abs(bounds0[0] - bounds0[1]) < EPS:
            return (-1, -1)
    
    #Then establish interval bounds 
    lowerB = [0, a0]
    upperB = [a0, 100]
    
    #Upper Bound
    a1 = a0
    step = STEP
    pos = POS.perf
    while abs(upperB[0] - upperB[1]) > EPS2:
        a1 = a1 + step
        accepted = pVals[pVals['Final_tstat'] > a1]
        
        #Determine position of alpha 
        if len(accepted.index) == 0:
            pos = POS.big
        else:
            pos = POS.perf
        
        #Determine what alpha to check next 
        if pos == POS.perf:
            upperB[0] = a1
            if a1 + abs(step * FACTOR2) >= upperB[1]:
                step = abs(a1 - upperB[1])/FACTOR2
            else:
                step = abs(step * FACTOR2) 
        elif pos == POS.big:
            upperB[1] = a1
            if (a1 - abs(step * FACTOR2)) <= upperB[0]:
                step = -1 * abs(a1 - upperB[0])/FACTOR2
            else:
                step = -1 * abs(step * FACTOR2) 
        else:
            assert False

    #Lower Bound
    a2 = a0
    if a2 - STEP > 1e-20:
        step = STEP
    else: 
        step = a2/FACTOR2 
    pos = POS.perf
    while abs(lowerB[0] - lowerB[1]) > EPS2:
        a2 = a2 - step
        accepted = pVals[pVals['Final_tstat'] > a2]
        
        #Determine position of alpha 
        accepted_sets = [str_2_pcp(a) for a in list(accepted.index)]
        causal_preds = set.intersection(*accepted_sets)
        if len(causal_preds) == 0:
            pos = POS.small 
        else:
            pos = POS.perf       
        
        #Determine what alpha to check next 
        if pos == POS.perf:
            lowerB[1] = a2
            if a2 - abs(step * FACTOR2) <= lowerB[0]:
                step = abs(a2 - lowerB[0])/FACTOR2
            else:
                step = abs(step * FACTOR2) 
        elif pos == POS.small:
            lowerB[0] = a2
            if (a1 + abs(step * FACTOR2)) >= lowerB[1]:
                step = -1 * abs(a2 - lowerB[1])/FACTOR2
            else:
                step = -1 * abs(step * FACTOR2) 
        else:
            assert False
    
    #Check if interval is too close to 0 to be meaningful 
    if a2 < amin: 
        return (-1, -1)
        
    #Establish 0-padding to interval
    interval = abs(a1 - a2)/5
    
    assert (a2 < a0) and (a0 < a1)
    
    return (max(0, a2 - interval), a1 + interval)


def max_alpha(pVals, arange, eps=1000): 
    '''Given a computed range of CP returning alphas (maybe with interval) and pvals for exp, return highest CP returning alpha'''
    ctr = arange[1]
    while ctr > arange[0]:
        accepted = pVals[pVals['Final_tstat'] > ctr]
        if len(accepted.index) > 0:
            return ctr
        else:
            ctr = ctr - (arange[1] - arange[0])/eps
    return -1

# # File Generation

In [35]:
#Collect all files appropiate to each unique identifier 
rawres_files= []
for f in os.listdir(expdir):
    if ('rawres_' in f):
        rawres_files.append(f)

# Parameters

In [27]:
def open_pvals(filename):
    try:
        pvals = json.load(open(filename, 'rb'))
        del pvals["()"]
    except:
        return None
    pvals = pd.DataFrame.from_dict(pvals, orient='index')
    return pvals
    

In [54]:
#Generate Alphas 
NUM_POINTS = 100
MIN_ALPHA = 1e-4

alphas = {}
for fname in rawres_files:
    pvals = open_pvals(os.path.join(expdir, fname))
    if pvals is None:
        continue
    f, d, s, e, rd = get_hps_from_rawres(fname) 
    arange = alpha_tune(pvals, MIN_ALPHA)
    alphas[(f, rd, s, d, e)] = [x for x in arange] + [NUM_POINTS] + [max_alpha(pvals, arange)]
#     if (f == '1') and (rd == '1000') and (s == '1000') and (d == 'adult') and (e == 'native-country'):
#         arange = alpha_tune(pvals, MIN_ALPHA)
#         print(max_alpha(pvals, arange))
#         assert False
    
alphas = pd.DataFrame(alphas).T
alphas.columns = ['start', 'stop', 'num_points', 'max_alpha']
alphas.index.names = ['feateng', 'reddata', 'seed', 'dataset', 'env']
alphas.head(1000)

start      stop  num_points  \
feateng reddata seed dataset env                                              
1       1000    1000 adult   marital-status -1.000000 -1.000000       100.0   
                             native-country  0.024094  0.055310       100.0   
                             occupation     -1.000000 -1.000000       100.0   
                             relationship   -1.000000 -1.000000       100.0   
                             workclass       0.369009  0.545112       100.0   
                147  adult   marital-status -1.000000 -1.000000       100.0   
                             native-country  0.019480  0.041083       100.0   
                             occupation     -1.000000 -1.000000       100.0   
                             relationship   -1.000000 -1.000000       100.0   
                             workclass       1.491107  3.266724       100.0   
                256  adult   marital-status -1.000000 -1.000000       100.0   
                             native-country  0.000000  0.455729       100.0   
                             occupation     -1.000000 -1.000000       100.0   
                             relationship   -1.000000 -1.000000       100.0   
                             workclass       0.655691  1.459787       100.0   
                304  adult   marital-status -1.000000 -1.000000       100.0   
                             native-country  0.076628  0.771014       100.0   
                             occupation     -1.000000 -1.000000       100.0   
                             relationship   -1.000000 -1.000000       100.0   
                             workclass       1.257858  2.119989       100.0   
                52   adult   marital-status -1.000000 -1.000000       100.0   
                             native-country  0.326949  1.454261       100.0   
                             occupation      0.000000  0.003753       100.0   
                             relationship   -1.000000 -1.000000       100.0   
                             workclass       0.948015  1.666993       100.0   
                587  adult   marital-status -1.000000 -1.000000       100.0   
                             native-country  2.267339  3.912793       100.0   
                             occupation      0.000220  0.000861       100.0   
                             relationship   -1.000000 -1.000000       100.0   
                             workclass       1.075438  3.057008       100.0   
...                                               ...       ...         ...   
12      7500    52   adult   marital-status -1.000000 -1.000000       100.0   
                             native-country -1.000000 -1.000000       100.0   
                             occupation     -1.000000 -1.000000       100.0   
                             relationship   -1.000000 -1.000000       100.0   
                             workclass       0.000000  0.034106       100.0   
                587  adult   marital-status -1.000000 -1.000000       100.0   
                             native-country  0.000000  0.005232       100.0   
                             occupation     -1.000000 -1.000000       100.0   
                             relationship   -1.000000 -1.000000       100.0   
                             workclass       0.000000  0.096235       100.0   
                784  adult   marital-status -1.000000 -1.000000       100.0   
                             native-country -1.000000 -1.000000       100.0   
                             occupation     -1.000000 -1.000000       100.0   
                             relationship   -1.000000 -1.000000       100.0   
                             workclass      -1.000000 -1.000000       100.0   
                8079 adult   marital-status -1.000000 -1.000000       100.0   
                             native-country -1.000000 -1.000000       100.0   
                             occupation     -1.000000 -1.000000       100.0   
                   

# Generate pVals  Results

In [106]:
x_axis = {}  #x,y values  for plot of alpha vs #CPs 
y_axis = {}
CPid_results = {}  #Stores CPids of each expierment 

for fname in rawres_files:
    #Identify exp 
    f, d, s, e, rd = get_hps_from_rawres(fname)  
    unid = '{}_{}_{}_{}_{}'.format(f,d,s,e,rd)
    
    #Load pvals
    pvals = open_pvals(os.path.join(expdir, fname))
    if pvals is None:
        continue
    
    #Create entries in all results data structures 
    x_axis[(f, s, d, e, rd)] = []
    y_axis[(f, s, d, e, rd)] = []
    CPid_results[(f, s, d, e, rd)] = Counter()

    ###Generate Results
    
    ##For results dependent on all alphas returning CPs 
    start, stop, num_points = alphas.loc[f, rd, s, d, e][0], alphas.loc[f, rd, s, d, e][1], alphas.loc[f, rd, s, d, e][2]
    for a in np.linspace(start, stop, num_points): 
        accepted = pvals[pvals['Final_tstat'] > a]
        if len(accepted.index) > 100000:
            raise ValueError('too many subsets: {}'.format(len(accepted.index)))

        accepted_sets = list(accepted.index)
        accepted_sets = [str_2_pcp(a) for a in accepted_sets]
        if len(accepted_sets) > 0:
            pcps = set.intersection(*accepted_sets)
        else:
            pcps = set([])
        
        #Store Number of Accepted Sets 
        x_axis[(f, s, d, e, rd)].append(a)
        if len(accepted_sets) == 0:
            y_axis[(f, s, d, e, rd)].append(0)
        else:
            y_axis[(f, s, d, e, rd)].append(len(set.intersection(*accepted_sets)))

        #Store Causal predictors  
        for pcp in pcps: 
            CPid_results[(f, s, d, e, rd)].update({pcp:1})

    ##For results dependant on only max CP-retuning alpha


In [74]:
alphas.loc['12', '10000', '1000', 'adult', 'marital-status'] [3]

-1.0

In [76]:
###Test Code for getting the coefficients 
def hack_pcp2str()

for fname in rawres_files[0:1]:
    #Get Pvals for environment of interest
    pvals = open_pvals(os.path.join(expdir, fname))
    if pvals is None:
        continue
    f, d, s, e, rd = get_hps_from_rawres(fname) 
    #Get alpha of interest and comptue its associated CPs 
    max_a = alphas.loc[f, rd, s, d, e][3]
    accepted = pvals[pvals['Final_tstat'] > max_a]
    accepted_sets = list(accepted.index)
    accepted_sets = [str_2_pcp(a) for a in accepted_sets]
    
    #For the given pval file, 
print(str(accepted_sets[0]))
pvals.head()   

{"'age'", "'capital-gain'"}


,"('marital-status_married',)","('marital-status_neither',)","('marital-status_DUMmY',)",Final_tstat
"('age', 'capital-gain')",2.575294e-289,0.0,0.0,0.0
"('age', 'capital-gain', 'capital-loss')",7.206265e-284,0.0,0.0,0.0
"('age', 'capital-gain', 'capital-loss', 'hours-per-week')",1.211407e-220,0.0,0.0,0.0
"('age', 'capital-gain', 'hours-per-week')",1.403706e-223,0.0,0.0,0.0
"('age', 'capital-loss')",5.663133e-296,0.0,0.0,0.0


# Generate Coefficients Results 

In [109]:
def get_data_regressors(atts, sub, ft_eng, data):
    '''From a given subset of attributes being predicted on and the attributes
    dictionary with the original columns, extract all coluns to predict on from
    dataset

    :param atts - dictionary of attributes, {att, [one-hot col list of at vals]}
    :param sub - subset of atts being predicted on
    :param ft_eng - [which mods applicable]
    '''
    orig_regressors = [atts[cat] for cat in sub]
    orig_regressors = [item for sublist in orig_regressors for item in sublist if '_DUMmY' not in item]
    #Now have all the actual one-hot columns in dataset

    if not ft_eng:
        return orig_regressors

    one_regressors = []
    two_regressors = []
    if 1 in ft_eng: #Assumes only single-col vals are squared
        sq_regressors = [col for col in data.columns if '_sq' in col]
        for r in orig_regressors:
            for r_sq in sq_regressors:
                if r in r_sq:
                    one_regressors.append(r_sq)

    if 2 in ft_eng:
        x_regressors = [col for col in data.columns if '_x_' in col]
        for r in [com for com in combinations(orig_regressors, 2) \
            if (com[0].split('_')[0] != com[1].split('_')[0])]:

            for x_reg in x_regressors:
                if ((r[0] in x_reg) and (r[1] in x_reg)):
                    two_regressors.append(x_reg)

    return orig_regressors + one_regressors + two_regressors

In [144]:
NUM_COEFFS = 25
dataset_fname = '/Users/RobertAdragna/Documents/School/Fourth_Year/ESC499-Thesis/codebases/causal_discovery/data/adult.csv'
coeffs = {}

for fname in rawres_files:
    #Identify exp 
    f, d, s, e, rd = get_hps_from_rawres(fname)  
    unid = 'causalcoeffs_{}_{}_{}_{}_{}'.format(f,d,s,e,rd)
    key_id = (f, s, d, e, rd)
    
    #Record Result
    coeffs[key_id] = {}
    
    #Load pvals
    pvals = open_pvals(os.path.join(expdir, fname))
    if pvals is None:
        continue
        
    #Get the Causal Predictors 
    accepted = pvals[pvals['Final_tstat'] > alphas.loc[f, rd, s, d, e]['max_alpha']]
    accepted_sets = [str_2_pcp(a) for a in list(accepted.index)]
    causal_preds = set.intersection(*accepted_sets)
        
    #Get Data 
    data, y_all, d_atts = dp.adult_dataset_processing(dataset_fname, \
                              [int(c) for c in f], reduce_dsize=int(rd), \
                              estrat_red=1, \
                              testing=0)
    
    #Get Environment Info 
    env_atts = [d_atts[e]]
    e_ins_store = {}
    for env in itertools.product(*env_atts):
        dummy_envs = []
        live_envs = []
        for att in env:
            if '_DUMmY' in att:
                dummy_envs = [d for d in d_atts[att.split('_')[0]] if d != att]
            else:
                live_envs.append(att)

        #Compute e_in without error
        if not dummy_envs:
            e_in = ((data[live_envs] == 1)).all(1)
        elif not live_envs:
            e_in = ((data[dummy_envs] == 0)).all(1)
        else:
            e_in = ((data[live_envs] == 1).all(1) & (data[dummy_envs] == 0).all(1))
        e_ins_store[str(env)] = e_in
    
    #Linear regression on all data
    regressors = get_data_regressors(d_atts, [x.strip('"').strip("''") for x in causal_preds], \
                                     [int(c) for c in f], data)
    x_s = data[list(itertools.chain(regressors))]
    if x_s.shape[1] == 0: 
#         print('{} has no CPs'.format(unid))
        continue
    
    #Store Causal coefficients for each env 
    for env, e_in in e_ins_store.items():
        c = (LinearRegression(fit_intercept=False).fit(x_s.loc[e_in].values, y_all.loc[e_in].values)).coef_[0]
        n = list(x_s.columns) 
        assert len(c) == len(n)
        coeffs[key_id][env] = sorted(zip(c, n), reverse=True, key=lambda x: abs(x[0]))[:NUM_COEFFS]

    c = (LinearRegression(fit_intercept=False).fit(x_s.values, y_all.values)).coef_[0]    
    n = list(x_s.columns) 
    coeffs[key_id]['final'] = sorted(zip(c, n), reverse=True, key=lambda x: abs(x[0]))[:NUM_COEFFS]

In [155]:
import pprint
pprint.pprint(coeffs[('1', '1000', 'adult', 'native-country', '1000')])

{"('native-country_DUMmY',)": [(0.42534476454243536, 'education_Prof-school'),
                               (0.36078019238864906, 'relationship_spouse'),
                               (0.30362389413357393, 'education_Masters'),
                               (-0.28698233273129853, 'race_Black'),
                               (-0.2704377127574141, 'race_Asian-Pac-Islander'),
                               (-0.24711191888010461, 'race_White'),
                               (-0.2458245486118799, 'occupation_agriculture'),
                               (0.24352813454458286, 'education_Doctorate'),
                               (-0.22273998750841228, 'education_9th'),
                               (0.20634808761479512, 'education_Bachelors'),
                               (-0.18619347928862207, 'occupation_transport'),
                               (-0.1681589815537739, 'education_5th-6th'),
                               (-0.15952462713328922, 'occupation_machine'),
             

# Save Results

In [143]:
pickle.dump(x_axis, open(os.path.join(savedir, 'x_axis'), 'wb'))
pickle.dump(y_axis, open(os.path.join(savedir, 'y_axis'), 'wb'))
pickle.dump(CPid_results, open(os.path.join(savedir, 'CPid_results'), 'wb'))
pickle.dump(coeffs, open(os.path.join(savedir, 'coeffs'), 'wb'))

# Appendix

## CALIBRATION

In [ ]:
# #Plot Accepted subsets vs Alpha for specified hyperparams 

# #fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(w*5, int(l/w)*5)) #Note - is +2 for reason


# for exp in itertools.product(feateng, dataset, seed, environment):
#     for fname in rawres_files:
#         f, d, s, e, rd = get_hps_from_rawres(fname)
#         if (f == exp[0]) and (d == exp[1]) and (s == exp[2]) and (e == exp[3]) and (rd == exp[4]):
#             unid = '{}_{}_{}_{}'.format(f,d,s,e, rd)
#             try:
#                 pvals = json.load(open(os.path.join(expdir, fname), 'rb'))
#                 del pvals["()"]
#             except:
#                 continue
#             pvals = pd.DataFrame.from_dict(pvals, orient='index')
            
#             start, stop, num_points = alphas.loc[f, rd, s, d, e][0], alphas.loc[f, rd, s, d, e][1], alphas.loc[f, rd, s, d, e][2]
#             for a in np.linspace(start, stop, num_points): 
#                 accepted = pvals[pvals['Final_tstat'] > a]
#                 if len(accepted.index) == 0:
#                     print(a, unid, 0, 'null')
#                 elif len(accepted.index) < 1000:
#                     accepted_sets = list(accepted.index)
#                     accepted_sets = [str_2_pcp(a) for a in accepted_sets]
#                     print(a, unid, len(accepted.index), set.intersection(*accepted_sets))
#                 else:
#                     print(a, unid, len(accepted.index), 'too_many_intersections')
            
    
#     print('#####################################')